In [25]:
import pandas as pd
import os

In [57]:
import pandas as pd

# List of file names
files = [
    "MSFT_final_predictions.csv",
    "MARA_final_predictions.csv",
    "JNJ_final_predictions.csv",
    "KO_final_predictions.csv",
    "TSLA_final_predictions.csv",
    "WMT_final_predictions.csv",
    "JPM_final_predictions.csv",
    "IBM_final_predictions.csv",
    "V_final_predictions.csv"
]

# Empty list to store data
all_last_records = []

# Loop through each file and extract last 2 records
for file in files:
    try:
        df = pd.read_csv(file)
        last_two = df.tail(2)  # Get last 2 rows
        last_two["File"] = file  # Add a column to track file name
        all_last_records.append(last_two)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Combine all records into a single DataFrame
if all_last_records:
    result_df = pd.concat(all_last_records, ignore_index=True)
    # Save to CSV
    result_df.to_csv("last_two_records.csv", index=False)
    print("Saved last 2 records from each file into 'last_two_records.csv'")
else:
    print("No records extracted.")


Saved last 2 records from each file into 'last_two_records.csv'


C:\Users\32468\AppData\Local\Temp\ipykernel_29944\80383125.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_two["File"] = file  # Add a column to track file name
C:\Users\32468\AppData\Local\Temp\ipykernel_29944\80383125.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_two["File"] = file  # Add a column to track file name
C:\Users\32468\AppData\Local\Temp\ipykernel_29944\80383125.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [ ]:
def calculate_percentage_changes(filename): 
    if not os.path.exists(filename):
        print(f"File not found: {filename}")
        return None

    # Read the CSV file
    df = pd.read_csv(filename)

    # Check if 'Prediction' column exists
    if 'Prediction' not in df.columns:
        print(f"'Prediction' column not found in {filename}. Columns available: {df.columns}")
        return None

    values = df['Prediction'].tolist()

    # Ensure there are enough values to calculate percentage changes
    if len(values) < 2:
        print(f"Not enough data in {filename}")
        return None

    # Calculate percentage changes
    percentage_changes = [(values[i] - values[i-1]) / values[i-1] * 100 
                         for i in range(1, len(values))]

    # Separate increases and decreases
    increases = [p for p in percentage_changes if p > 0]
    decreases = [p for p in percentage_changes if p < 0]

    # Calculate statistics
 # Calculate statistics with inting
 
    avg_increase = int(sum(increases) / len(increases)) if increases else 0

    avg_decrease = int(sum(decreases) / len(decreases)) if decreases else 0

    # Extract ticker from filename
    ticker = filename.split('_')[0]

    return {
        'Ticker': ticker,
        'Avg Increase': f"{avg_increase:}%",
        'Avg Decrease': f"{avg_decrease:}%"
    }


In [47]:
# Create a list to store results
all_results = []

# Process each file
for file in files:
    results = calculate_percentage_changes(file)
    if results:
        all_results.append(results)

# Convert results to DataFrame and save
if all_results:
    results_df = pd.DataFrame(all_results)
    print(results_df)  # Print instead of display() for non-Jupyter environments
    results_df.to_csv('percentage_changes.csv', index=False)
    print("Results saved to 'percentage_changes.csv'")
else:
    print("No valid data found.")

  Ticker Avg Increase Avg Decrease
0   MSFT          24%         -18%
1   MARA         214%         -47%
2    JNJ          15%          -4%
3     KO           7%          -3%
4   TSLA          67%         -34%
5    WMT          33%          -5%
6    JPM          39%          -7%
7    IBM          22%          -1%
8      V          12%          -1%
Results saved to 'percentage_changes.csv'


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

# Portfolio definitions
portfolios = {
    "Short-Term (Volatile Portfolio)": ["MSFT", "TSLA", "MARA"],
    "Moderate-Term (Stable Portfolio)": ["TSLA", "JNJ", "IBM", "WMT", "JPM"],
    "Long-Term (Moderate Portfolio)": ["KO", "WMT", "V"]
}

# Investment budgets
investment_budgets = {
    "Short-Term (Volatile Portfolio)": 500,
    "Moderate-Term (Stable Portfolio)": 8000,
    "Long-Term (Moderate Portfolio)": 30000
}

# Financial Goals
goals = {
    "Short-Term": {
        "iPhone 16": 1300,
        "Vacation Trip": 3500
    },
    "Moderate-Term": {
        "Car": 41950,
        "House": 330000
    },
    "Long-Term": {
        "Retirement": 1000000
    }
}

# Stock performance data (estimates)
all_results = {
    "MSFT": {"Avg Increase": 24, "Avg Decrease": -18},
    "MARA": {"Avg Increase": 214, "Avg Decrease": -47},
    "JNJ": {"Avg Increase": 15, "Avg Decrease": -4},
    "KO": {"Avg Increase": 7, "Avg Decrease": -3},
    "TSLA": {"Avg Increase": 67, "Avg Decrease": -34},
    "WMT": {"Avg Increase": 33, "Avg Decrease": -5},
    "JPM": {"Avg Increase": 39, "Avg Decrease": -7},
    "IBM": {"Avg Increase": 22, "Avg Decrease": -1},
    "V": {"Avg Increase": 12, "Avg Decrease": -1},
}

# Compound interest growth years
def calculate_years_to_goal(initial, growth_rate, goal):
    if growth_rate <= 0:
        return "∞"
    years = np.log(goal / initial) / np.log(1 + growth_rate)
    return round(years, 2)

# Dividend yield fetcher
def get_dividend_yield(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.info.get("dividendYield", 0) or 0
    except:
        return 0



# Compile all portfolio data
portfolio_results = []

for portfolio_name, stocks in portfolios.items():
    investment = investment_budgets[portfolio_name]
    investment_per_stock = investment / len(stocks)
    total_growth = 0
    total_dividends = 0

    for stock in stocks:
        avg_increase = all_results[stock]["Avg Increase"] / 100
        avg_decrease = all_results[stock]["Avg Decrease"] / 100
        net_growth = (avg_increase + avg_decrease) / 2
        dividend_yield = get_dividend_yield(stock)

        total_growth += net_growth
        total_dividends += dividend_yield

    avg_net_growth = total_growth / len(stocks)
    avg_dividend_yield = total_dividends / len(stocks)
    effective_growth = avg_net_growth + avg_dividend_yield

    # Determine goals per category
    if "Short-Term" in portfolio_name:
        goal_set = goals["Short-Term"]
    elif "Moderate-Term" in portfolio_name:
        goal_set = goals["Moderate-Term"]
    elif "Long-Term" in portfolio_name:
        goal_set = goals["Long-Term"]
    else:
        goal_set = {}

    goal_years = {}
    for goal_name, goal_value in goal_set.items():
        years_needed = calculate_years_to_goal(investment, effective_growth, goal_value)
        goal_years[goal_name] = years_needed

    portfolio_results.append({
        "Portfolio": portfolio_name,
        "Investment Amount": investment,
        "Avg Growth (no dividend)": f"{avg_net_growth*100:.2f}%",
        "Avg Dividend Yield": f"{avg_dividend_yield*100:.2f}%",
        "Effective Growth Rate": f"{effective_growth*100:.2f}%",
        **goal_years,
    })

# Create final DataFrame
df = pd.DataFrame(portfolio_results)
pd.set_option('display.max_columns', None)
print(df)

# Save to file
df.to_csv("portfolio_goals_with_feedback.csv", index=False)
print("\nFile saved: portfolio_goals_with_feedback.csv")


                          Portfolio  Investment Amount  \
0   Short-Term (Volatile Portfolio)                500   
1  Moderate-Term (Stable Portfolio)               8000   
2    Long-Term (Moderate Portfolio)              30000   

  Avg Growth (no dividend) Avg Dividend Yield Effective Growth Rate  \
0                   34.33%             29.33%                63.67%   
1                   12.50%            181.80%               194.30%   
2                    7.17%            158.67%               165.83%   

   iPhone 16  Vacation Trip  \
0       1.94           3.95   
1        NaN            NaN   
2        NaN            NaN   

                                     Tips & Feedback   Car  House  Retirement  
0  ⚡ High risk, high reward. Good for quick goals...   NaN    NaN         NaN  
1  📈 Balanced growth with some safety. Great for ...  1.54   3.45         NaN  
2  💰 Excellent for long-term compounding. Strong ...   NaN    NaN        3.59  

✅ File saved: portfolio_goals_with_f

In [55]:
import pandas as pd
import numpy as np

# Investment amount
initial_investment = 20000

# Financial goals
goal_car = 41950
goal_house = 330000
goal_retirement = 1000000

# Provided stock data
all_results = {
    "MSFT": {"Avg Increase": 24, "Avg Decrease": -18},
    "MARA": {"Avg Increase": 214, "Avg Decrease": -47},
    "JNJ": {"Avg Increase": 15, "Avg Decrease": -4},
    "KO": {"Avg Increase": 7, "Avg Decrease": -3},
    "TSLA": {"Avg Increase": 67, "Avg Decrease": -34},
    "WMT": {"Avg Increase": 33, "Avg Decrease": -5},
    "JPM": {"Avg Increase": 39, "Avg Decrease": -7},
    "IBM": {"Avg Increase": 22, "Avg Decrease": -1},
    "V": {"Avg Increase": 12, "Avg Decrease": -1},
}

# Define portfolios
portfolios = {
    "Volatile Portfolio": ["MSFT", "TSLA", "MARA"],
    "Stable Portfolio": ["TSLA","JNJ", "IBM", "WMT", "JPM"],
    "Moderate Portfolio": ["KO", "WMT", "V"]
}

# Function to calculate years needed for a goal
def calculate_years_to_goal(initial, growth_rate, goal):
    """Calculate the number of years to reach a financial goal using compound interest formula."""
    if growth_rate <= 0:
        return "∞"  # If the growth rate is non-positive, goal will never be reached
    years = np.log(goal / initial) / np.log(1 + growth_rate)
    return round(years, 2)

# Process each portfolio
portfolio_results = []
for portfolio_name, stocks in portfolios.items():
    total_growth = 0
    total_increase = 0
    total_decrease = 0

    # Calculate average net growth for the portfolio
    for stock in stocks:
        avg_increase = all_results[stock]["Avg Increase"] / 100
        avg_decrease = all_results[stock]["Avg Decrease"] / 100
        net_growth = (avg_increase + avg_decrease) / 2  # Averaging increase and decrease
        total_growth += net_growth
        total_increase += avg_increase
        total_decrease += avg_decrease

    avg_portfolio_growth = total_growth / len(stocks)  # Portfolio's average growth rate
    avg_portfolio_increase = (total_increase / len(stocks)) * 100  # Convert to percentage
    avg_portfolio_decrease = (total_decrease / len(stocks)) * 100  # Convert to percentage

    # Calculate years required for financial goals
    years_to_car = calculate_years_to_goal(initial_investment, avg_portfolio_growth, goal_car)
    years_to_house = calculate_years_to_goal(initial_investment, avg_portfolio_growth, goal_house)
    years_to_retirement = calculate_years_to_goal(initial_investment, avg_portfolio_growth, goal_retirement)

    # Store portfolio results
    portfolio_results.append({
        "Portfolio": portfolio_name,
        "Avg Increase": f"{avg_portfolio_increase:.2f}%",
        "Avg Decrease": f"{avg_portfolio_decrease:.2f}%",
        "Net Growth Rate": f"{avg_portfolio_growth * 100:.2f}%",
        "Years to Car": years_to_car,
        "Years to House": years_to_house,
        "Years to Retirement": years_to_retirement,
    })

# Convert to DataFrame and save
portfolio_df = pd.DataFrame(portfolio_results)
print(portfolio_df)  # Print for verification

# Save to CSV
portfolio_df.to_csv("portfolio_years_to_goals.csv", index=False)
print("Results saved to 'portfolio_years_to_goals.csv'")


            Portfolio Avg Increase Avg Decrease Net Growth Rate  Years to Car  \
0  Volatile Portfolio      101.67%      -33.00%          34.33%          2.51   
1    Stable Portfolio       35.20%      -10.20%          12.50%          6.29   
2  Moderate Portfolio       17.33%       -3.00%           7.17%         10.70   

   Years to House  Years to Retirement  
0             9.5                13.25  
1            23.8                33.21  
2            40.5                56.52  
Results saved to 'portfolio_years_to_goals.csv'
